<a href="https://colab.research.google.com/github/mikexcohen/Statistics_book/blob/main/stats_ch12_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modern statistics: Intuition, Math, Python, R
## Mike X Cohen (sincxpress.com)
### https://www.amazon.com/dp/B0CQRGWGLY
#### Code for chapter 12 (correlation)

---

# About this code file:

### This notebook will reproduce most of the figures in this chapter (some figures were made in Inkscape), and illustrate the statistical concepts explained in the text. The point of providing the code is not just for you to recreate the figures, but for you to modify, adapt, explore, and experiment with the code.

### Solutions to all exercises are at the bottom of the notebook.

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# import module from scipy (for cosine similarity)
from scipy import spatial

# pandas and seaborn for the exercises
import pandas as pd
import seaborn as sns

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     })

# Figure 12.1: Example of scatter plot showing correlated data

In [ ]:
# Number of sibling pairs
num_pairs = 40

# Simulate ratings for brothers
brothers_ratings = np.random.randint(1,11,num_pairs)

# Simulate correlated ratings for sisters based on brothers' ratings
noise = np.random.normal(0,2,num_pairs)  # some random noise
sisters_ratings = brothers_ratings + noise  # sister's ratings are brother's ratings plus some noise

# Ensure ratings are within bounds 1 and 10
sisters_ratings = np.clip(np.round(sisters_ratings),1,10)

# correlation
r = stats.pearsonr(brothers_ratings,sisters_ratings)[0]

# Create a scatter plot of the data
plt.figure(figsize=(6,5))
plt.plot(brothers_ratings, sisters_ratings,'ks',markersize=10,markerfacecolor=(.7,.7,.7))
plt.title(f'Death metal preferences in siblings (r={r:.2f})',loc='center')
plt.xlabel("Brothers' ratings")
plt.ylabel("Sisters' ratings")
plt.xlim(0,11)
plt.ylim(0,11)
plt.grid(True)

plt.tight_layout()
plt.savefig('cor_death.png')
plt.show()

# Figure 12.2: Different correlation coefficients

In [ ]:
# correlation values
rs = [ 1,.7,.2,0,0,0,-.2,-.7,-1 ]

# sample size
N = 188


# start the plotting!
_,axs = plt.subplots(3,3,figsize=(8,8.5))

for r,ax,i in zip(rs,axs.flatten(),range(9)):

  # generate data
  x = np.random.randn(N)
  y = x*r + np.random.randn(N)*np.sqrt(1-r**2)

  # exceptions for r=0
  if i==3:
    x = np.cos(np.linspace(0,2*np.pi-2*np.pi/N,N))
    y = np.sin(np.linspace(0,2*np.pi-2*np.pi/N,N))
  elif i==4:
    x = np.linspace(-2,2,N)
    y = x**2
  elif i==5:
    x = np.linspace(-2,2,N//2)
    y = np.concatenate((x,-x),0)
    x = np.concatenate((x,x),0)

  # empirical correlation
  rho = np.corrcoef(x,y)[0,1]

  # plot
  ax.plot(x,y,'ko',markersize=10,markerfacecolor=(.7,.7,.7),alpha=.3)
  ax.set(xticks=[],yticks=[])
  ax.set_title(f'r = {rho:.2f}',loc='center')


plt.tight_layout()
plt.savefig('cor_variousRs.png')
plt.show()

# Figure 12.3: Same correlation, different slopes

In [ ]:
N = 100

# Dataset 1
x1 = np.random.normal(100,10,N)
y1 = .3*x1 + np.random.randn(N)*3
slope1,intercept1,r1,_,_ = stats.linregress(x1,y1)

# Dataset 2
x2 = np.random.normal(10,1,N) + np.mean(x1)
y2 = 3*x2 + np.random.randn(N)*3
slope2,intercept2,r2,_,_ = stats.linregress(x2,y2)

# x-axis limits
xmin,xmax = np.min(x1)-5,np.max(x1)+5

# Plot datasets and their regression lines
_,axs = plt.subplots(1,2,figsize=(10,4))

axs[0].plot(x1,y1,'ko',markersize=10,markerfacecolor=(.3,.3,.3),alpha=.3)
axs[0].plot(x1,intercept1 + slope1*x1,'k')
axs[0].set_title(fr'$\bf{{A}}$) Slope={slope1:.2f}, r={r1:.2f}')
axs[0].set(xlim=[xmin,xmax])

axs[1].plot(x2,y2,'ks',markersize=10,markerfacecolor=(.3,.3,.3),alpha=.3)
axs[1].plot(x2, intercept2 + slope2*x2,'k')
axs[1].set_title(fr'$\bf{{B}}$) Slope={slope2:.2f}, r={r2:.2f}')
axs[1].set(xlim=[xmin,xmax])

plt.tight_layout()
plt.savefig('cor_fitlineR.png')
plt.show()

# numpy vs scipy

In [ ]:
# vector input
x = np.random.randn(50)
y = x + np.random.randn(len(x))/2

usingScipy = stats.pearsonr(x,y)
usingNumpy = np.corrcoef(x,y)

print('scipy.stats.pearsonr:')
print(usingScipy)

print(' ')
print('numpy.corrcoef:')
print(usingNumpy)

In [ ]:
# matrix input (features by observations)
X = np.vstack((x[None,:],y[None,:]))

# usingScipy = stats.pearsonr(X) ## gives an error!
usingNumpy = np.corrcoef(X)

print('numpy.corrcoef:')
print(usingNumpy)

In [ ]:
# to get a matrix of R and p values:

R = np.zeros((2,2))
P = np.zeros((2,2))

# Calculate Pearson correlation for each pair of variables
for i in range(2):
  for j in range(2):
    R[i,j], P[i,j] = stats.pearsonr(X[i,:],X[j,:])


print('Correlation matrix:')
print(R)

print('')
print('Associated p-values:')
print(P)

# Note about this code cell: you don't actually need to compute all matrix elements;
# you can compute only the upper-triangle and then copy the results to the lower triangle.

# Creating correlated data

In [ ]:
# Method 1 (quick&dirty but effective)
x = np.random.randn(40)
y = x + np.random.randn(len(x))
r = stats.pearsonr(x,y).statistic

plt.plot(x,y,'ks')
plt.title(f'r = {r:.3f}',loc='center')
plt.show()

In [ ]:
# method 2 (more control over the correlation)
r = .4
x = np.random.randn(50)
y = np.random.randn(len(x))
y = x*r + y*np.sqrt(1-r**2)

rr = stats.pearsonr(x,y).statistic

plt.plot(x,y,'ks')
plt.title(f'r = {rr:.3f}',loc='center')
plt.show()

In [ ]:
# method 3: multivariate
C = np.array([ [1,.4],[.4,1] ])
means = np.zeros(2)

X = np.random.multivariate_normal(means,C,50)
r = np.corrcoef(X.T)

plt.plot(X[:,0],X[:,1],'ks')
plt.title(f'r = {r[0,1]:.3f}',loc='center')
plt.show()

# Figure 12.6:  Anscobe's quartet

In [ ]:

anscombe = np.array([
     # series 1     series 2      series 3       series 4
    [10,  8.04,    10,  9.14,    10,  7.46,      8,  6.58, ],
    [ 8,  6.95,     8,  8.14,     8,  6.77,      8,  5.76, ],
    [13,  7.58,    13,  8.76,    13, 12.74,      8,  7.71, ],
    [ 9,  8.81,     9,  8.77,     9,  7.11,      8,  8.84, ],
    [11,  8.33,    11,  9.26,    11,  7.81,      8,  8.47, ],
    [14,  9.96,    14,  8.10,    14,  8.84,      8,  7.04, ],
    [ 6,  7.24,     6,  6.13,     6,  6.08,      8,  5.25, ],
    [ 4,  4.26,     4,  3.10,     4,  5.39,      8,  5.56, ],
    [12, 10.84,    12,  9.13,    12,  8.15,      8,  7.91, ],
    [ 7,  4.82,     7,  7.26,     7,  6.42,      8,  6.89, ],
    [ 5,  5.68,     5,  4.74,     5,  5.73,     19, 12.50, ]
    ])


# plot data and correlations
fig,ax = plt.subplots(2,2,figsize=(8,5))
ax = ax.ravel()

for i in range(4):

  # plot the points
  ax[i].plot(anscombe[:,i*2],anscombe[:,i*2+1],'ko',markersize=10,markerfacecolor=(.7,.7,.7))

  # compute the corrs
  corr_p = stats.pearsonr(anscombe[:,i*2],anscombe[:,i*2+1])[0]
  corr_s = stats.spearmanr(anscombe[:,i*2],anscombe[:,i*2+1])[0]

  # update the axis
  ax[i].set(xticks=[],yticks=[])
  ax[i].set_title(f'$r_p = {corr_p:.2f},  r_s = {corr_s:.2f}$',loc='center')


plt.tight_layout()
plt.savefig('cor_anscobe.png')
plt.show()

# Toy covariance example

In [ ]:
# raw scores
h = np.array([74,63,58,70])
s = np.array([ 4, 7, 2, 9])
N = len(h)

# demeaned
hd = h-np.mean(h)
sd = s-np.mean(s)

cov = np.sum(hd*sd) / (N-1)
cov, hd*sd

# Figure 12.7: Kandall tau

In [ ]:
# The data
bro = np.array([ 1,2,3,4,5 ])
sis = np.array([ 2,1,4,5,3 ])

# the correlation
k = stats.kendalltau(bro,sis)

# band names (lol)
bands = [ 'Unicorn Apocalypse',
          "Satan's Fluffy Bunnies",
          'Demonic Dishwashers',
          'Vampiric Vegetarians',
          'Zombie Zucchini Zephyr' ]

# the plot
_,ax = plt.subplots(1,figsize=(6,5))
ax.plot(bro,sis,'ks',markersize=30,markerfacecolor=(.9,.9,.9))
ax.set(xlabel="Brother's ratings",ylabel="Sister's ratings",
       yticks=range(1,6),xticks=range(1,6),
       xlim=[.5,5.5],ylim=[.5,5.5])
ax.grid()
ax.set_title(fr'$\tau$ = {k[0]:.2f}',loc='center')



# band names
for i in range(len(bro)):

  # band names in plot markers
  ax.text(bro[i],sis[i],bands[i][0],size=20,
           weight='bold',va='center',ha='center')

  # raw data
  ax.text(9,i+1,bands[i],ha='right',va='center')
  ax.text(9.5,i+1,bro[i],ha='center',va='center')
  ax.text(10,i+1,sis[i],ha='center',va='center')

# column labels
ax.text(9,5.5,'Band name',ha='right',va='center',weight='bold')
ax.text(9.5,5.5,'Bro',ha='center',va='center',weight='bold')
ax.text(10,5.5,'Sis',ha='center',va='center',weight='bold')


# plt.tight_layout()
plt.savefig('cor_kendall.png')
plt.show()

# Figure 12.8: Statistical significance of r based on n

In [ ]:
# simulation ranges
rs = np.linspace(.1,.8,53) # r values
ns = np.arange(10,511,step=25) # sample sizes

# compute the matrix of t-values
num = rs[:,None]*np.sqrt(ns-2)
den = 1-rs[:,None]**2
tmat = num/den


## show the matrix!
fig,ax = plt.subplots(1,figsize=(4,6))

cax = ax.imshow(tmat,vmin=2,vmax=10,aspect='auto',cmap='gray',
           extent=[ns[0],ns[-1],rs[0],rs[-1]],origin='lower')
ax.set(xlabel='Sample size',ylabel='Correlation strength')

# and make it look a bit nicer
fig.colorbar(cax,orientation='horizontal',pad=.02,ax=ax,location='top')
ax.spines[['right','top']].set_visible(True)


plt.tight_layout()
plt.savefig('cor_tvals.png')
plt.show()

# Figure 12.9: Fisher-z transform on uniform data

In [ ]:
# "correlation" data and its Fisher transform
r = np.random.uniform(-1,1,size=1000)
fish_r = np.arctanh(r)



# now for plotting
fig,axs = plt.subplots(1,2,figsize=(10,4))

# histograms
axs[0].hist(fish_r, bins='fd', color=(.3,.3,.3),alpha=1, label='F(r)')
axs[0].hist(r,bins='fd', color='k', alpha=.5, label='r')
axs[0].legend(loc='upper right')
axs[0].set_xlabel('Value')
axs[0].set_ylabel('Frequency')
axs[0].set_title(r'$\bf{A}$)  Histograms of r and F(r)')

# scatter plot to visualize the transform
axs[1].plot(r,fish_r,'ko',markersize=10,alpha=.5,markerfacecolor=(.8,.8,.8))
axs[1].set_xlabel('r')
axs[1].set_ylabel('F(r)')
axs[1].set_title(r'$\bf{B}$)  Scatter plot of r vs. F(r)')

plt.tight_layout()
plt.savefig('cor_fisherFull.png')
plt.show()

# Figure 12.10: Fisher-z transform on numbers close to zero

In [ ]:
# Same as above but with simulated H0 coefficients

# random data
X = np.random.randn(100,45)

# compute the correlation matrix and extract the unique r values
R = np.corrcoef(X.T)
utri = np.triu(R,k=1)
r = utri[utri!=0]

# fisher transform
fish_r = np.arctanh(r)


# now for plotting
fig,axs = plt.subplots(1,2,figsize=(10,4))

# histograms
axs[0].hist(fish_r, bins='fd', color=(.3,.3,.3),alpha=1, label='F(r)')
axs[0].hist(r,bins='fd', color='k', alpha=.5, label='r')
axs[0].legend(loc='upper right')
axs[0].set_xlabel('Value')
axs[0].set_ylabel('Frequency')
axs[0].set_title(r'$\bf{A}$)  Histograms of r and F(r)')

# scatter plot to visualize the transform
axs[1].plot(r,fish_r,'ko',markersize=10,alpha=.5,markerfacecolor=(.8,.8,.8))
axs[1].set_xlabel('r')
axs[1].set_ylabel('F(r)')
axs[1].set_title(r'$\bf{B}$)  Scatter plot of r vs. F(r)')

plt.tight_layout()
plt.savefig('cor_fisherReal.png')
plt.show()

# Figure 12.11: Subgroups paradox

In [ ]:
# initializations
n = 20 # sample points per group
offsets = [2, 3.5, 5] # mean offsets

allx = np.array([])
ally = np.array([])

s = 'so^' # marker shapes

# generate and plot data
plt.figure(figsize=(5,6))
for datai in range(3):

  # generate data
  x = np.linspace(offsets[datai]-1,offsets[datai]+1,n)
  y = -x/3 + np.mean(x) + np.random.randn(n)/3

  # subgroup correlation
  r,p = stats.pearsonr(x,y)

  # plot
  plt.plot(x,y,s[datai],color=(datai/3,datai/3,datai/3),
           markersize=14,alpha=.7,label=f'r={r:.2f}, p={p:.2f}')

  # gather the data into one array
  allx = np.append(allx,x)
  ally = np.append(ally,y)



# % now correlate the groups
[r,p] = stats.pearsonr(allx,ally)
plt.title(f'Total r={r:.2f}, p={p:.3f}',loc='center')

plt.xticks([])
plt.yticks([])
plt.legend()

plt.tight_layout()
plt.savefig('cor_simpsons.png')
plt.show()

# Cosine similarity

In [ ]:
# variables
x = np.array([ 1,2,3,4 ])
y = np.array([ 1,2,3,4 ])
z = np.array([ 101,102,103,104])

# correlations
r_xy = np.corrcoef(x,y)[0,1]
r_xz = np.corrcoef(x,z)[0,1]

# cosine similarities
c_xy = 1-spatial.distance.cosine(x,y)
c_xz = 1-spatial.distance.cosine(x,z)


# print out the results
print(f'r_xy: {r_xy:.3f}')
print(f'c_xy: {c_xy:.3f}')
print('')
print(f'r_xz: {r_xz:.3f}')
print(f'c_xz: {c_xz:.3f}')

# Exercise 1

In [ ]:
# two random correlated variables
v = np.random.randn(10)
w = v + np.random.randn(len(v))

### correlation using mean-centered dot products
# mean-center
vm = v-np.mean(v)
wm = w-np.mean(w)

# dot products
r_me = np.dot(vm,wm) / np.sqrt(np.dot(vm,vm)*np.dot(wm,wm))


### correlation using numpy
r_np = np.corrcoef(v,w)[0,1]


# print results
print(f'r from np.corr: {r_np:.3f}')
print(f'r from np.dot : {r_me:.3f}')

# Exercise 2

In [ ]:
# create random data
N = 43
r = .4
x = np.random.randn(N)
y = np.random.randn(N)
y = x*r + y*np.sqrt(1-r**2)

# r,p from numpy
r_np = np.corrcoef(x,y)[0,1]
t = r_np*np.sqrt(N-2) / (1-r_np**2)
p_np = stats.t.sf(np.abs(t),N-2) * 2  # times 2 for a two-sided test

# r,p from scipy
r_sp,p_sp = stats.pearsonr(x,y)

# print correlation values
print(f'r (p) from numpy: {r_np:.4f} ({p_np:.4f})')
print(f'r (p) from scipy: {r_sp:.4f} ({p_sp:.4f})')

In [ ]:
# now in a loop

# I wrote a function to output the two p-values,
# which makes the experiment code below easier to read.
def getpvals(x,y):

  # r,p from numpy
  r_np = np.corrcoef(x,y)[0,1]
  t = r_np*np.sqrt(len(x)-2) / (1-r_np**2)
  p_np = stats.t.sf(np.abs(t),len(x)-2) * 2  # times 2 for a two-sided test

  # r,p from scipy
  r_sp,p_sp = stats.pearsonr(x,y)

  return p_np,p_sp


# range of correlation values
rvals = np.linspace(0,.99,40)

# results matrix
pvalues = np.zeros((len(rvals),2))


## run the experiment
for ri in range(len(rvals)):

  # create the data
  x = np.random.randn(44)
  y = np.random.randn(44)
  y = x*rvals[ri] + y*np.sqrt(1-rvals[ri]**2)

  # get the two p-values
  pvalues[ri,:] = getpvals(x,y)



# plot
plt.figure(figsize=(8,5))
plt.plot(rvals,np.log(pvalues[:,1]),'ko',markersize=12,markerfacecolor=(.9,.9,.9),label='scipy')
plt.plot(rvals,np.log(pvalues[:,0]),'ks',markersize=12,markerfacecolor=(.7,.7,.7),label='numpy',zorder=-1)
plt.legend()
plt.xlabel('Correlation value (r)')
plt.ylabel('log(p)')

plt.tight_layout()
plt.savefig('cor_ex2.png')
plt.show()

# Exercise 3

In [ ]:
# matrix of p-values

N = 10000 # observations
M = 15 # features

# data matrix
X = np.random.randn(N,M)

# correlation matrix
R = np.corrcoef(X.T)

# confirm that it's the right shape
print(f'Correlation matrix shape: {R.shape}')

In [ ]:
# compute the t-values
Tnum = R*np.sqrt(N-2)
Tden = 1-R**2 + np.finfo(float).eps # adding a tiny number to avoid n/0

T = Tnum / Tden

# compute the p-values
P = stats.t.sf(T,N-2)

# visualize all matrices
fig,axs = plt.subplots(1,3,figsize=(10,5))

cax = axs[0].imshow(R,vmin=-.1,vmax=.1,cmap='gray')
axs[0].set_title(r'$\bf{A}$)  R matrix')
c = fig.colorbar(cax,fraction=.046,pad=.04); c.ax.tick_params(labelsize=10)

cax = axs[1].imshow(T,vmin=-2,vmax=2,cmap='gray')
axs[1].set_title(r'$\bf{B}$)  T matrix')
c = fig.colorbar(cax,fraction=.046,pad=.04); c.ax.tick_params(labelsize=10)

cax = axs[2].imshow(P,vmin=0,vmax=.05,cmap='gray')
axs[2].set_title(r'$\bf{C}$)  P matrix')
c = fig.colorbar(cax,fraction=.046,pad=.04); c.ax.tick_params(labelsize=10)

# properties common to all axes
for a in axs:
  a.set(xticks=[],yticks=[],xlabel='Features',ylabel='Features')
  a.spines[['right','top']].set_visible(True)


plt.tight_layout()
plt.savefig('cor_ex3.png')
plt.show()

# Exercise 4

In [ ]:
# create the sigma matrix
Sigma = np.std(X,ddof=1,axis=0)
Sigma = np.diag(Sigma)

# compute C from R
C_me = Sigma@R@Sigma # from formula
C_np = np.cov(X.T,ddof=1) # from numpy

# check for equality (these should all be True)
(C_np-C_me)

In [ ]:
# Now compute R from C
invSigma = 1/np.std(X,ddof=1,axis=0)
invSigma = np.diag(Sigma)

R_me = invSigma@C_np@invSigma


# check for equality (these should all be True)
(R-R_me) < 1e-16

# Exercise 5

In [ ]:
# simulation parameters
N = 1000  # observations
M =   20  # features

# number of repetitions
numReps = 30


# initialize data lists
alldata = np.zeros((M,N))
corrmats = np.zeros((M,M,numReps+1))

# "pure" data
covars = np.linspace(-1,1,M)[:,None]
dataOG = np.random.randn(N) * covars


# random noise in each repetition
for idx in range(numReps):

  # this run's data
  thisdata = dataOG + np.random.randn(M,N)*5

  # its correlation matrix
  corrmats[:,:,idx] = np.corrcoef(thisdata)

  # sum the data
  alldata += thisdata



# correlation of data average
corrmats[:,:,-1] = np.corrcoef(alldata)


### plotting
fig,axs = plt.subplots(1,3,figsize=(12,5))

axs[0].imshow(covars@covars.T,vmin=-.3,vmax=.3,cmap='gray')
axs[0].set_title(r'$\bf{A}$)  Ground truth')

axs[1].imshow(np.mean(corrmats[:,:,:-1],axis=2),vmin=-.3,vmax=.3,cmap='gray')
axs[1].set_title(r'$\bf{B}$)  Ave. of correlations')

cax = axs[2].imshow(corrmats[:,:,-1],vmin=-.3,vmax=.3,cmap='gray')
axs[2].set_title(r'$\bf{C}$)  Correlation of ave.')

cbar_ax = fig.add_axes([.91,.22,.015,.55])
cbar = plt.colorbar(cax,cax=cbar_ax)


# properties common to all axes
for a in axs:
  a.set(xticks=[],yticks=[],xlabel='Features',ylabel='Features')
  a.spines[['right','top']].set_visible(True)

# plt.tight_layout()
plt.savefig('cor_ex5.png')
plt.show()

# Exercise 6

In [ ]:
# re-initialize data lists
alldata = np.zeros((M,N))


### run the experiment
for idx in range(numReps):

  # this run's data (only 'covars' is constant across repetitions)
  thisdata = np.random.randn(N)*covars + np.random.randn(M,N)

  # its correlation matrix
  corrmats[:,:,idx] = np.corrcoef(thisdata)

  # store the data
  alldata += thisdata



# correlation of data average
corrmats[:,:,-1] = np.corrcoef(alldata)


### plotting
fig,axs = plt.subplots(1,3,figsize=(12,5))

axs[0].imshow(covars@covars.T,vmin=-.3,vmax=.3,cmap='gray')
axs[0].set_title(r'$\bf{A}$)  Ground truth')

axs[1].imshow(np.mean(corrmats[:,:,:-1],axis=2),vmin=-.3,vmax=.3,cmap='gray')
axs[1].set_title(r'$\bf{B}$)  Ave. of correlations')

cax = axs[2].imshow(corrmats[:,:,-1],vmin=-.3,vmax=.3,cmap='gray')
axs[2].set_title(r'$\bf{C}$)  Correlation of ave.')

cbar_ax = fig.add_axes([.91,.22,.015,.55])
cbar = plt.colorbar(cax,cax=cbar_ax)


# properties common to all axes
for a in axs:
  a.set(xticks=[],yticks=[],xlabel='Features',ylabel='Features')
  a.spines[['right','top']].set_visible(True)

# plt.tight_layout()
plt.savefig('cor_ex6.png')
plt.show()

# Exercise 7

In [ ]:
samplesize = 30
nSamples = 23

corrs = np.zeros(nSamples)
tres  = np.zeros(2)

# loop over experiments
for ni in range(nSamples):

  # create the data
  x = np.random.randn(samplesize)
  y = np.random.randn(samplesize)
  y = x*.1 + y*np.sqrt(1-.1**2)

  # correlation
  corrs[ni] = np.corrcoef(x,y)[0,1]

# now for a t-test on r values
tres[0] = stats.ttest_1samp(corrs,0).statistic
tres[1] = stats.ttest_1samp(np.arctanh(corrs),0).statistic

# critical t-value
tCrit = stats.t.isf(.05/2,samplesize-2)


print(f't-value from "raw" coefficients   : {tres[0]:.4f}')
print(f't-value from Fisher-z coefficients: {tres[1]:.4f}')
print(f'Critical t-value for p<.05        : {tCrit:.4f}')

In [ ]:
# now for a wider range of r values
rs = np.linspace(.01,.5,21)
samplesize = 30
nSamples = 23

corrs = np.zeros((len(rs),nSamples))
tres  = np.zeros((len(rs),2))

# critical t (doesn't depend on the population r or sample size)
tCrit = stats.t.isf(.05/2,samplesize-2)


# run the experiment!
for ri,r in enumerate(rs):

  # loop over experiments
  for ni in range(nSamples):

    # create the data
    x = np.random.randn(samplesize)
    y = np.random.randn(samplesize)
    y = x*r + y*np.sqrt(1-r**2)

    # correlation
    corrs[ri,ni] = np.corrcoef(x,y)[0,1]

  # now for a t-test on r values
  tres[ri,0] = stats.ttest_1samp(corrs[ri,:],0).statistic
  tres[ri,1] = stats.ttest_1samp(np.arctanh(corrs[ri,:]),0).statistic



## plot
_,axs = plt.subplots(1,2,figsize=(10,4))
axs[0].plot(rs,np.mean(corrs,axis=1),'ks',markersize=10,markerfacecolor=(.6,.6,.6))
axs[0].plot([rs[0],rs[-1]],[rs[0],rs[-1]],'--',color=(.8,.8,.8),zorder=-3)
axs[0].set(xlabel='Theoretical correlation',ylabel='Empirical correlation')
axs[0].set_title(r'$\bf{A}$)  Theoretical vs. empirical r')

axs[1].plot(rs,tres[:,0],'ks',markersize=10,markerfacecolor=(.6,.6,.6),label='"Raw"')
axs[1].plot(rs,tres[:,1],'ko',markersize=10,markerfacecolor=(.3,.3,.3),label='Fisher-z')
axs[1].plot(rs,np.full(len(rs),tCrit),'--',color=(.8,.8,.8),zorder=-3,label='Critical t')
axs[1].legend()
axs[1].set(xlabel='Theoretical correlation',ylabel='T values')
axs[1].set_title(r"$\bf{B}$)  T-values ($r's\neq 0$)")


plt.tight_layout()
plt.savefig('cor_ex7.png')
plt.show()

# Exercise 8

In [ ]:
# generate some correlated random data
x = np.random.randn(40)
y = x + np.random.randn(len(x))

# manual cosine similarity
cs_num = sum(x*y)
cs_den = np.sqrt(sum(x*x)) * np.sqrt(sum(y*y))
cs_me = cs_num / cs_den

# using the  distance function in the scipy.spatial library
# Note: using this function is confusing, because it computes *distance* although we want *similarity*.
# Fortunately, the two are simple inverses, so one is 1- the other.
cs_sp = 1-spatial.distance.cosine(x,y)


print(f'Manual result: {cs_me:.3f}')
print(f'Scipy.spatial: {cs_sp:.3f}')

# Exercise 9

In [ ]:
#  range of requested correlation coefficients
rs = np.linspace(-1,1,100)

# sample size
N = 500


# initialize output matrix
corrs = np.zeros((len(rs),2))


# loop over a range of r values
for ri in range(len(rs)):

  # generate data
  x = np.random.randn(N)
  y = x*rs[ri] + np.random.randn(N)*np.sqrt(1-rs[ri]**2)

  # mean de-centering
  x = x-10

  # compute correlation
  corrs[ri,0] = np.corrcoef(x,y)[0,1]

  # compute cosine similarity
  corrs[ri,1] = 1-spatial.distance.cosine(x,y)



## visualize the results
_,axs = plt.subplots(1,2,figsize=(10,4.5))

axs[0].plot(rs,corrs[:,0],'ks',markersize=10,markerfacecolor=(.5,.5,.5),alpha=.5,label='Correlation')
axs[0].plot(rs,corrs[:,1],'ko',markersize=10,markerfacecolor=(.9,.9,.9),alpha=.5,label='Cosine sim.')
axs[0].legend()
axs[0].set(xlabel='Requested correlation',ylabel=r'Empirical $r$ or $S_C$')
axs[0].set_title(r'$\bf{A}$)  Correlation and cosine sim.')

axs[1].plot(corrs[:,0],corrs[:,1],'ks',markersize=10,markerfacecolor=(.2,.2,.2),alpha=.5)
axs[1].axhline(y=0,color='gray',linestyle='--')
axs[1].axvline(x=0,color='gray',linestyle='--')
axs[1].set(xlabel='Correlation',ylabel='Cosine similarity')
axs[1].set_title(rf'$\bf{{B}}$)  r={np.corrcoef(corrs.T)[1,0]:.2f}')


plt.tight_layout()
plt.savefig('cor_ex9.png')
plt.show()

# Exercise 10

In [ ]:
# import the data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight','Acceleration','Model Year','Origin','Car Name']

data = pd.read_csv(url,delim_whitespace=True,names=column_names, na_values="?")
data

In [ ]:
# examine distributions

# include only numerical variables
data_numerical = data.drop(columns=['Car Name','Origin'])

# draw histograms with seaborn
fig,axs = plt.subplots(2,4,figsize=(12,6))
for a,column in zip(axs.flatten(),data_numerical.columns):
  sns.histplot(data=data_numerical, x=column, ax=a, color=(.7,.7,.7))

# only 7 columns, so switch off the empty 8th :P
axs[-1,-1].axis('off')

plt.tight_layout()
plt.savefig('cor_ex10b.png')
plt.show()

In [ ]:
# Create a correlation matrix
R = data_numerical.corr(method='spearman')

plt.figure(figsize=(8,6))
sns.heatmap(R, annot=True, cmap='coolwarm',vmin=-1,
            xticklabels=R.columns,yticklabels=R.columns)

plt.title('Correlation matrix (Spearman)',loc='center',weight='bold')
plt.tight_layout()
plt.savefig('cor_ex10c.png')
plt.show()

# Exercise 11

In [ ]:
# Calculate R and P matrices from Spearman correlation
R,P = stats.spearmanr(data_numerical,nan_policy='omit')

# store as dataframes for seaborn plotting (note: "df" here means "dataframe")
R_df = pd.DataFrame(R, columns=data_numerical.columns, index=data_numerical.columns)
P_df = pd.DataFrame(P, columns=data_numerical.columns, index=data_numerical.columns)

# Bonferroni correction [ formula is (M*(M-1))/2 ]
num_comparisons = (data_numerical.shape[1]*(data_numerical.shape[1]-1)) / 2
bonferroni_thresh = .05 / num_comparisons
significant = P_df < bonferroni_thresh


# Create a matrix of annotations
annot_array = R_df.astype(str).values

# loop through all elements of the matrix and create a string to display
for i in range(R_df.shape[0]):
  for j in range(R_df.shape[1]):

    # the string depends on the significance
    if not significant.iloc[i,j]:
      # if non-significant, just the correlation coefficient
      annot_array[i,j] = f'{R_df.iloc[i, j]:.2f}'
    else:
      # if significant, add an asterisk to the coefficient
      annot_array[i,j] = f'{R_df.iloc[i, j]:.2f}*'

    # don't need to report the diagonals (trivially=1)
    if i==j:
      annot_array[i,j] = ''


## now show the image
plt.figure(figsize=(8,6))
sns.heatmap(R_df,annot=annot_array,fmt='s',cmap='coolwarm',vmin=-1,
            xticklabels=R_df.columns,yticklabels=R_df.columns)

plt.title('Correlation matrix (*p<.05 corrected)',loc='center',weight='bold')
plt.tight_layout()
plt.savefig('cor_ex11.png')
plt.show()